In [3]:
"""
Importing python packages and and functions.  
"""

import os
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw
from Diffusion_Functions import frame_adjust, MSD_Deff_Calcs, prettify2, fillin2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
name = '100nm_RED_PEG_NPs_in_0_4p_agarose'

logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)
Tplot = '{}_Tplot'.format(name)
T2plot = '{}_T2plot'.format(name)

conversion = dict()

conversion[1] = (0.10, 9.91, 1)
conversion[2] = (0.10, 9.91, 1)
conversion[3] = (0.10, 9.91, 1)
conversion[4] = (0.10, 9.91, 1)
conversion[5] = (0.10, 9.91, 1)

cut = 100
great = 10000
filtered = False
new_method = True
tofilt = np.array([])

totvids = 1

trajectory = dict()
trajectory[1] = np.genfromtxt('./RED_PEG_NPs_0_4p_agarose1.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('./RED_PEG_NPs_0_4p_agarose2.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('./RED_PEG_NPs_0_4p_agarose3.csv',
            delimiter =",")
trajectory[4] = np.genfromtxt('./RED_PEG_NPs_0_4p_agarose4.csv',
            delimiter =",")
trajectory[5] = np.genfromtxt('./RED_PEG_NPs_0_4p_agarose5.csv',
            delimiter =",")

for num in range(1, totvids + 1):
    trajectory[num]=np.delete(trajectory[num],0,1)

parts = dict()
tots = dict()
newtots = dict()
newtots[0] = 0
tlen = dict()
tlength = dict()
tlength[0] = 0

for num in range(1, totvids + 1):
    tots[num] = trajectory[num][-1, 0].astype(np.int64)
    parts[num] = tots[num]
    counter = 1
    newtots[num] = newtots[num-1] + tots[num]
    
    tlen[num] = trajectory[num].shape[0]
    tlength[num] = tlength[num-1] + tlen[num]

placeholder = np.zeros((tlength[totvids], 11))

for num in range(1, totvids + 1):
    placeholder[tlength[num-1]:tlength[num], :] = trajectory[num]
    placeholder[tlength[num-1]:tlength[num], 0] = placeholder[tlength[num-1]:tlength[num], 0] + newtots[num-1]

In [3]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.
    
    Input: trajectory dataset from MOSAIC tracking software read into a numpy array
    Output: modified numpy array with missing frames filled in.
    """

    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [4]:
rawframes = placeholder[:, 1]
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)
time = frames / conversion[1][1]

x = dict()
y = dict()

xs = dict()
ys = dict()

M1x = dict() # MSD dictionaries (without shifting)
M1y = dict()
M2xy = dict()

SM1x = dict() # Shifted MSD dictionaries.
SM1y = dict()
SM2xy = dict()

SD1x = dict() # Shifted diffusion coefficient dictionaries.
SD1y = dict()
SD2xy = dict()

In [5]:
dataset = dict()
rawdataset = np.zeros(placeholder.shape)
particles = placeholder[:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = placeholder[:, :]

fixed = np.zeros(placeholder.shape)
fixed[:, 0:2] = rawdataset[:, 0:2]
fixed[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
fixed[:, 4] = conversion[1][2] * rawdataset[:, 4]

for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (fixed[min1:max1+1, 0:5])

In [6]:
I = dict()

for num in range(1, total1):
    #Construct x, y, z
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]
   
    xs[num] = np.zeros(frames.shape[0])
    xs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 2]
    ys[num] = np.zeros(frames.shape[0])
    ys[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 3]

In [7]:
cutoff = cut

x1 = dict()
y1 = dict()

xs1 = dict()
ys1 = dict()

fifties = 0
nones = 0

for num in range(1, total1):
    if np.count_nonzero(x[num]) < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        x1[num - nones] = x[num]
        y1[num - nones] = y[num]
        
        xs1[num - nones] = xs[num]
        ys1[num - nones] = ys[num]

x = x1
y = y1

xs = xs1
ys = ys1

for num in range(1, fifties):
    xs[num] = ma.masked_equal(xs[num], 0)
    ys[num] = ma.masked_equal(ys[num], 0)

    x[num] = ma.masked_equal(x[num], 0)
    y[num] = ma.masked_equal(y[num], 0)

In [8]:
total1 = fifties + 1

print('Total particles after merging datasets and filtering short trajectories:', fifties)

Total particles after merging datasets and filtering short trajectories: 88


In [9]:
xymask = dict()

tau_m = 100
frame_m = 446 #atm I can't go lower than the actual value.

# Intermediates for new method
m1xa = dict()
m1ya = dict()
m2xya = dict()

a1x = np.zeros((total1, frame_m))
a1y = np.zeros((total1, frame_m))
a2xy = np.zeros((total1, frame_m))

exists = os.path.isfile('M1x_{}.csv'.format(name))

if exists == False:
    last_size = 1
    with open('M1x_{}.csv'.format(name), "wb") as f_handle:
        pass
    with open('M1y_{}.csv'.format(name), "wb") as f_handle:
        pass
    with open('M2xy_{}.csv'.format(name), "wb") as f_handle:
        pass
else:
    with open('M1x_{}.csv'.format(name), "rb") as f_handle:
        reader = np.genfromtxt(f_handle, delimiter = ",")
        data = list(reader)
        last_size = data[0].shape[0]

for num in range(last_size, total1):

    M1x[num] = np.zeros(frame_m)
    M1x[num][0:frame_m] = 0
    M1y[num] = np.zeros(frame_m)
    M1y[num][0:frame_m] = 0
    M2xy[num] = np.zeros(frame_m)
    M2xy[num][0:frame_m] = 0
    
    SM1x[num] = np.zeros(frame_m)
    SM1y[num] = np.zeros(frame_m)
    SM2xy[num] = np.zeros(frame_m)
    
    SD1x[num] = np.zeros(frame_m)
    SD1y[num] = np.zeros(frame_m)
    SD2xy[num] = np.zeros(frame_m)
    
    I[num] = np.nonzero(x[num])[0]
    first = I[num][0]
    last = I[num][-1] + 1
    startx = x[num][first]
    starty = y[num][first]
    
    if new_method == True:
        
        m1xa[num] = dict()
        m1ya[num] = dict()
        m2xya[num] = dict()
        
        for num1 in range(1, tau_m):

            tau = num1
        
            m1xa[num][num1] = np.zeros(frame_m - tau)
            m1ya[num][num1] = np.zeros(frame_m - tau)
            m2xya[num][num1] = np.zeros(frame_m - tau)

            for num2 in range(0, frame_m - tau):
                m1xa[num][num1][num2] = (xs[num][num2 + tau] - xs[num][num2])**2
                m1ya[num][num1][num2] = (ys[num][num2 + tau] - ys[num][num2])**2
                m2xya[num][num1][num2] = m1xa[num][num1][num2] + m1ya[num][num1][num2]
                
            m1xa[num][num1] = ma.masked_invalid(m1xa[num][num1])
            m1ya[num][num1] = ma.masked_invalid(m1ya[num][num1])
            m2xya[num][num1] = ma.masked_invalid(m2xya[num][num1])

            SM1x[num][num1] = np.mean(m1xa[num][num1])
            SM1y[num][num1] = np.mean(m1ya[num][num1])
            SM2xy[num][num1] = np.mean(m2xya[num][num1])
            
#         SM1x[num] = ma.masked_invalid(SM1x[num])
#         SM1y[num] = ma.masked_invalid(SM1y[num])
#         SM2xy[num] = ma.masked_invalid(SM2xy[num])

        M1x[num][first:] = SM1x[num][:frame_m-first]
        M1y[num][first:] = SM1y[num][:frame_m-first]
        M2xy[num][first:] = SM2xy[num][:frame_m-first]
        
        M1x[num] = ma.masked_invalid(M1x[num])
        M1y[num] = ma.masked_invalid(M1y[num])
        M2xy[num] = ma.masked_invalid(M2xy[num])
        
    else:  

        for num1 in range(first, last):
            M1x[num][num1] = (x[num][num1] - startx)**2
            M1y[num][num1] = (y[num][num1] - starty)**2
            M2xy[num][num1] = (x[num][num1] - startx)**2 + (y[num][num1] - starty)**2

    M1x[num] = ma.masked_equal(M1x[num], 0)
    M1y[num] = ma.masked_equal(M1y[num], 0)
    M2xy[num] = ma.masked_equal(M2xy[num], 0)
    
    if new_method == False:

        if M2xy[num][~M2xy[num].mask].shape[0] != 0:
            fir = np.nonzero(M2xy[num])[0][0] - 1
            firl = M1x[num][fir:].shape[0]
        else:
            fir = 0
            firl = 46

        SM1x[num][0:firl] = M1x[num][fir:] #Fills in shifted MSD dictionaries
        SM1y[num][0:firl] = M1y[num][fir:]
        SM2xy[num][0:firl] = M2xy[num][fir:]

#     SM1x[num] = ma.masked_equal(SM1x[num], 0)
#     SM1y[num] = ma.masked_equal(SM1y[num], 0)
#     SM2xy[num] = ma.masked_equal(SM2xy[num], 0)
    
    xs[num] = ma.masked_equal(xs[num], 0)
    ys[num] = ma.masked_equal(ys[num], 0)    
    
#     SD1x[num][1:] = SM1x[num][1:]/(2*time[1:frame_m]) # Calculating diffusion coefficients.
#     SD1y[num][1:] = SM1y[num][1:]/(2*time[1:frame_m])
#     SD2xy[num][1:] = SM2xy[num][1:]/(4*time[1:frame_m])

#     SD1x[num] = ma.masked_equal(SD1x[num], 0)
#     SD1y[num] = ma.masked_equal(SD1y[num], 0)
#     SD2xy[num] = ma.masked_equal(SD2xy[num], 0)
    
    a1x[num-1, :] = SM1x[num]
    a1y[num-1, :] = SM1y[num]
    a2xy[num-1, :] = SM2xy[num]

    a1x[num] = ma.masked_equal(a1x[num], 0)
    a1y[num] = ma.masked_equal(a1y[num], 0)
    a2xy[num] = ma.masked_equal(a2xy[num], 0)

    a1x[num] = ma.masked_invalid(a1x[num])
    a1y[num] = ma.masked_invalid(a1y[num])
    a2xy[num] = ma.masked_invalid(a2xy[num])

    if num == 1:
        with open('M1x_{}.csv'.format(name), "wb") as f_handle:
            np.savetxt(f_handle, SM1x[num], delimiter = ",")
    
        with open('M1y_{}.csv'.format(name), "wb") as f_handle:
            np.savetxt(f_handle, SM1y[num], delimiter = ",")

        with open('M2xy_{}.csv'.format(name), "wb") as f_handle:
            np.savetxt(f_handle, SM2xy[num], delimiter = ",")
        
    else:
        with open('M1x_{}.csv'.format(name), "rb") as f_handle:
            old_one = np.genfromtxt(f_handle, delimiter =",")
        new = np.column_stack((old_one, SM1x[num]))
        with open('M1x_{}.csv'.format(name), "wb") as f_handle:
            np.savetxt(f_handle, new, delimiter = ",")
    
        with open('M1y_{}.csv'.format(name), "rb") as f_handle:
            old_one1 = np.genfromtxt(f_handle, delimiter =",")
        new1 = np.column_stack((old_one1, SM1y[num]))
        with open('M1y_{}.csv'.format(name), "wb") as f_handle:
            np.savetxt(f_handle, new1, delimiter = ",")

        with open('M2xy_{}.csv'.format(name), "rb") as f_handle:
            old_one2 = np.genfromtxt(f_handle, delimiter =",")
        new2 = np.column_stack((old_one2, SM2xy[num]))
        with open('M2xy_{}.csv'.format(name), "wb") as f_handle:
            np.savetxt(f_handle, new2, delimiter = ",")

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [10]:
with open('M1x_{}.csv'.format(name), "rb") as f_handle:
    test = np.genfromtxt(f_handle, delimiter =",")
with open('M1y_{}.csv'.format(name), "rb") as f_handle:
    test1 = np.genfromtxt(f_handle, delimiter =",")
with open('M2xy_{}.csv'.format(name), "rb") as f_handle:
    test2 = np.genfromtxt(f_handle, delimiter =",")

SM1x = dict()
SM1y = dict()
SM2xy = dict()

total_check = test2.shape[1]

for num in range(1, total_check):
    SM1x[num] = test[:, num - 1]
    SM1y[num] = test1[:, num - 1]
    SM2xy[num] = test2[:, num - 1]
    
    SM1x[num] = ma.masked_equal(SM1x[num], 0)
    SM1y[num] = ma.masked_equal(SM1y[num], 0)
    SM2xy[num] = ma.masked_equal(SM2xy[num], 0)
    
    SM1x[num] = ma.masked_invalid(SM1x[num])
    SM1y[num] = ma.masked_invalid(SM1y[num])
    SM2xy[num] = ma.masked_invalid(SM2xy[num])

    xymask[num] = SM2xy[num].recordmask
    xs[num] = ma.array(xs[num][:frame_m], mask = xymask[num])
    ys[num] = ma.array(ys[num][:frame_m], mask = xymask[num])

In [11]:
arM1x = np.zeros(SM1x[1].shape[0])
arM1y = np.zeros(SM1x[1].shape[0])
arM2xy = np.zeros(SM1x[1].shape[0])

arD1x = np.zeros(SM1x[1].shape[0])
arD1y = np.zeros(SM1x[1].shape[0])
arD2xy = np.zeros(SM1x[1].shape[0])

st_arM1x = np.zeros(SM1x[1].shape[0])
st_arM1y = np.zeros(SM1x[1].shape[0])
st_arM2xy = np.zeros(SM1x[1].shape[0])

st_arD1x = np.zeros(SM1x[1].shape[0])
st_arD1y = np.zeros(SM1x[1].shape[0])
st_arD2xy = np.zeros(SM1x[1].shape[0])

In [12]:
tots = total_check - 1
time2 = time

gM1x = dict()
gM1y = dict()
gM2xy = dict()

log_gM1x = dict()
log_gM1y = dict()
log_gM2xy = dict()

geoM1x = np.zeros(SM1x[1].shape[0])
geoM1y = np.zeros(SM1x[1].shape[0])
geoM2xy = np.zeros(SM1x[1].shape[0])

st_geoM1x = np.zeros(SM1x[1].shape[0])
st_geoM1y = np.zeros(SM1x[1].shape[0])
st_geoM2xy = np.zeros(SM1x[1].shape[0])

geoD1x = np.zeros(SM1x[1].shape[0])
geoD1y = np.zeros(SM1x[1].shape[0])
geoD2xy = np.zeros(SM1x[1].shape[0])

st_geoD1x = np.zeros(SM1x[1].shape[0])
st_geoD1y = np.zeros(SM1x[1].shape[0])
st_geoD2xy = np.zeros(SM1x[1].shape[0])

for num2 in range(0, SM1x[1].shape[0]):
    gM1x[num2+1] = np.zeros(tots)
    gM1y[num2+1] = np.zeros(tots)
    gM2xy[num2+1] = np.zeros(tots)
    
    for num in range(1, tots + 1):
        gM1x[num2+1][num-1] = SM1x[num][num2]
        gM1y[num2+1][num-1] = SM1y[num][num2]
        gM2xy[num2+1][num-1] = SM2xy[num][num2]
        
    gM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
    gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
    gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])

    gM1x[num2+1] = ma.masked_equal(gM1x[num2+1], 0)
    gM1y[num2+1] = ma.masked_equal(gM1y[num2+1], 0)
    gM2xy[num2+1] = ma.masked_equal(gM2xy[num2+1], 0)

    log_gM1x[num2+1] = np.log(gM1x[num2+1])
    log_gM1y[num2+1] = np.log(gM1y[num2+1])
    log_gM2xy[num2+1] = np.log(gM2xy[num2+1])
        
    geoM1x[num2] = stat.gmean(gM1x[num2+1])
    geoM1y[num2] = stat.gmean(gM1y[num2+1])
    geoM2xy[num2] = stat.gmean(gM2xy[num2+1])
    
    st_geoM1x[num2] = np.abs(geoM1x[num2]-np.exp(np.mean(np.log(gM1x[num2+1]))-np.std(np.log(gM1x[num2+1]))/np.sqrt(gM1x[num2+1].shape[0])))
    st_geoM1y[num2] = np.abs(geoM1y[num2]-np.exp(np.mean(np.log(gM1y[num2+1]))-np.std(np.log(gM1y[num2+1]))/np.sqrt(gM1y[num2+1].shape[0])))
    st_geoM2xy[num2] = np.abs(geoM2xy[num2]-np.exp(np.mean(np.log(gM2xy[num2+1]))-np.std(np.log(gM2xy[num2+1]))/np.sqrt(gM2xy[num2+1].shape[0])))

    geoD1x[num2] = geoM1x[num2]/time2[num2]
    geoD1y[num2] = geoM1y[num2]/time2[num2]
    geoD2xy[num2] = geoM2xy[num2]/time2[num2]

    arM1x[num2] = np.mean(gM1x[num2+1])
    arM1y[num2] = np.mean(gM1y[num2+1])
    arM2xy[num2] = np.mean(gM2xy[num2+1])

    st_arM1x[num2] = np.std(gM1x[num2+1])
    st_arM1y[num2] = np.std(gM1y[num2+1])
    st_arM2xy[num2] = np.std(gM2xy[num2+1])

    arD1x[num2] = arM1x[num2]/time2[num2]
    arD1y[num2] = arM1y[num2]/time2[num2]
    arD2xy[num2] = arM2xy[num2]/time2[num2]

    st_arD1x[num2] = st_arM1x[num2]/time2[num2]
    st_arD1y[num2] = st_arM1y[num2]/time2[num2]
    st_arD2xy[num2] = st_arM2xy[num2]/time2[num2]

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [14]:
geoM2xy

array([         nan,   0.55496822,   1.06962517,   1.59006763,
         2.10710246,   2.63149007,   3.15606014,   3.6806718 ,
         4.20468949,   4.73007968,   5.25363441,   5.79269164,
         6.34116762,   6.89507153,   7.45108425,   8.00758634,
         8.5640081 ,   9.10558387,   9.65236492,  10.20988733,
        10.77477407,  11.36885666,  11.97805507,  12.61014335,
        13.22730054,  13.8556285 ,  14.49203694,  15.12520735,
        15.78540453,  16.45351261,  17.11372413,  17.79865903,
        18.4934844 ,  19.2156396 ,  19.93388789,  20.66798036,
        21.41135641,  22.15775475,  22.89690301,  23.64793907,
        24.42259093,  25.21132652,  25.98291209,  26.74724049,
        27.506089  ,  28.25542497,  29.02616819,  29.79765892,
        30.58866325,  31.40083282,  32.23092192,  33.03050453,
        33.83795741,  34.6331464 ,  35.47250727,  36.31798833,
        37.18948393,  38.06791055,  38.95459825,  39.85772339,
        40.79880412,  41.7461463 ,  42.67111841,  43.57

In [ ]:
np.savetxt('geoD2xy_{}.csv'.format(name), geoD2xy, delimiter=',')
np.savetxt('arD2xy_{}.csv'.format(name), arD2xy, delimiter=',')
#np.savetxt('FD2xy_{}.csv'.format(name), FD2xy, delimiter=',')

np.savetxt('geoM2xy_{}.csv'.format(name), geoM2xy, delimiter=',')
np.savetxt('arM2xy_{}.csv'.format(name), arM2xy, delimiter=',')
#np.savetxt('FM2xy_{}.csv'.format(name), FM2xy, delimiter=',')

np.savetxt('geoM1x_{}.csv'.format(name), geoM1x, delimiter=',')
np.savetxt('arM1x_{}.csv'.format(name), arM1x, delimiter=',')
np.savetxt('geoM1y_{}.csv'.format(name), geoM1y, delimiter=',')
np.savetxt('arM1y_{}.csv'.format(name), arM1y, delimiter=',')

In [16]:
def plot_Mcomp(Mgeo, Mar, MFBF, t1, t2, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Mgeo = Geometric average dataset
    Mar = Arithmetic average dataset
    MFBF = Frame-by-Frame dataset
    
    t1 = time array for Mgeo and Mar
    t2 time array for MFDF
    """
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t1, Mgeo, linewidth=10, label='Geometric', color='blue')
    ax.plot(t1, Mar, linewidth=10, label='Arithmetic', color='red')
    ax.plot(t2, MFBF, linewidth=10, label='Frame-by-Frame', color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.60, 0.66), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return Mar, Mgeo

In [18]:
"""
This block compares MSDs calculated by frame-by-frame, geometric, and arithmetic means.
"""

one1, bleh = plot_Mcomp(geoM2xy, arM2xy, geoM2xy, time2, time, 0, 0, 'MSD (um^2)', Cplot, 10.1, 80.1, 1, 10)
plt.show()

In [19]:
total1

89

In [20]:
total_check

88

In [22]:
divmod(total1, 16)

(5, 9)

In [23]:
total1

89

In [25]:
for i in range(0,10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [4]:
fake = dict()

for num in range(1, 201):
    fake[num] = np.zeros(200)


In [5]:
fake1 = dict()

for num in range(1, 201):
    fake1[num] = np.zeros(200)

In [6]:
fa = dict()
fa[1] = fake
fa[2] = fake1

fa[2][20]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [9]:
fa[1][1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [10]:
fa[1][1] = ma.masked_equal(fa[1][1], 0)
fa[1][1]

masked_array(data = [-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --],
             mask = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  Tr

In [19]:
fam = fa[1][1]
fam

masked_array(data = [-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --],
             mask = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  Tr

In [22]:
fam1 = fam.filled(0)
fam1

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [23]:
fa[1][2] = ma.masked_equal(fa[1][2], 1)

In [24]:
fa[1][2]

masked_array(data = [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.],
             mask = False,
       fill_value = 1.0)

In [34]:
SM1x = dict()
SM1y = dict()
SM2xy = dict()

size = 16
local_n = 5

for num in range(0, size):
    with open('pM1x_{}.csv'.format(num), "rb") as f_handle:
        interim = np.genfromtxt(f_handle, delimiter=",")
    for i in range(1, local_n+1):
        current = (local_n)*num + i
        SM1x[current] = interim[:, i-1]
    with open('pM1y_{}.csv'.format(num), "rb") as f_handle:
        interim1 = np.genfromtxt(f_handle, delimiter=",")
    for i in range(1, local_n+1):
        current = (local_n)*num + i
        SM1y[current] = interim1[:, i-1]
    with open('pM2xy_{}.csv'.format(num), "rb") as f_handle:
        interim2 = np.genfromtxt(f_handle, delimiter=",")
    for i in range(1, local_n+1):
        current = (local_n)*num + i
        SM2xy[current] = interim2[:, i-1]

In [35]:
SM2xy

{1: array([  0.        ,   0.58273494,   1.07957697,   1.53189259,
          2.00064197,   2.38036188,   2.74990659,   3.10065349,
          3.40890329,   3.73724868,   3.98715037,   4.24147292,
          4.57997258,   4.88744045,   5.19775216,   5.46378464,
          5.74298039,   6.04397367,   6.37521859,   6.62655974,
          6.86630737,   7.10720054,   7.31256323,   7.54319627,
          7.77365389,   7.97373597,   8.21474659,   8.50414596,
          8.79798843,   9.13217943,   9.45209226,   9.72864732,
          9.97714111,  10.29272985,  10.57344508,  10.85036579,
         11.24287902,  11.59700449,  11.91619717,  12.18277205,
         12.37103112,  12.66721665,  12.98649973,  13.25983372,
         13.68742794,  14.12055428,  14.5968454 ,  15.03873884,
         15.58673926,  16.14743196,  16.6300105 ,  17.13046281,
         17.73370962,  18.36478467,  19.02106693,  19.63163337,
         20.23828697,  20.93053982,  21.6310776 ,  22.34408278,
         22.9958657 ,  23.71674433,  

In [36]:
total_check = local_n*size

arM1x = np.zeros(SM1x[1].shape[0])
arM1y = np.zeros(SM1x[1].shape[0])
arM2xy = np.zeros(SM1x[1].shape[0])

arD1x = np.zeros(SM1x[1].shape[0])
arD1y = np.zeros(SM1x[1].shape[0])
arD2xy = np.zeros(SM1x[1].shape[0])

st_arM1x = np.zeros(SM1x[1].shape[0])
st_arM1y = np.zeros(SM1x[1].shape[0])
st_arM2xy = np.zeros(SM1x[1].shape[0])

st_arD1x = np.zeros(SM1x[1].shape[0])
st_arD1y = np.zeros(SM1x[1].shape[0])
st_arD2xy = np.zeros(SM1x[1].shape[0])

tots = total_check

gM1x = dict()
gM1y = dict()
gM2xy = dict()

log_gM1x = dict()
log_gM1y = dict()
log_gM2xy = dict()

geoM1x = np.zeros(SM1x[1].shape[0])
geoM1y = np.zeros(SM1x[1].shape[0])
geoM2xy = np.zeros(SM1x[1].shape[0])

st_geoM1x = np.zeros(SM1x[1].shape[0])
st_geoM1y = np.zeros(SM1x[1].shape[0])
st_geoM2xy = np.zeros(SM1x[1].shape[0])


for num2 in range(0, SM1x[1].shape[0]):
    gM1x[num2+1] = np.zeros(tots)
    gM1y[num2+1] = np.zeros(tots)
    gM2xy[num2+1] = np.zeros(tots)

    for num in range(1, tots+1):
        gM1x[num2+1][num-1] = SM1x[num][num2]
        gM1y[num2+1][num-1] = SM1y[num][num2]
        gM2xy[num2+1][num-1] = SM2xy[num][num2]

    gM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
    gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
    gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])

    gM1x[num2+1] = ma.masked_equal(gM1x[num2+1], 0)
    gM1y[num2+1] = ma.masked_equal(gM1y[num2+1], 0)
    gM2xy[num2+1] = ma.masked_equal(gM2xy[num2+1], 0)

    log_gM1x[num2+1] = np.log(gM1x[num2+1])
    log_gM1y[num2+1] = np.log(gM1y[num2+1])
    log_gM2xy[num2+1] = np.log(gM2xy[num2+1])

    geoM1x[num2] = stat.gmean(gM1x[num2+1])
    geoM1y[num2] = stat.gmean(gM1y[num2+1])
    geoM2xy[num2] = stat.gmean(gM2xy[num2+1])

    st_geoM1x[num2] = np.abs(geoM1x[num2]-np.exp(np.mean(np.log(gM1x[num2+1]))-np.std(np.log(gM1x[num2+1]))/np.sqrt(gM1x[num2+1].shape[0])))
    st_geoM1y[num2] = np.abs(geoM1y[num2]-np.exp(np.mean(np.log(gM1y[num2+1]))-np.std(np.log(gM1y[num2+1]))/np.sqrt(gM1y[num2+1].shape[0])))
    st_geoM2xy[num2] = np.abs(geoM2xy[num2]-np.exp(np.mean(np.log(gM2xy[num2+1]))-np.std(np.log(gM2xy[num2+1]))/np.sqrt(gM2xy[num2+1].shape[0])))


    arM1x[num2] = np.mean(gM1x[num2+1])
    arM1y[num2] = np.mean(gM1y[num2+1])
    arM2xy[num2] = np.mean(gM2xy[num2+1])

    st_arM1x[num2] = np.std(gM1x[num2+1])
    st_arM1y[num2] = np.std(gM1y[num2+1])
    st_arM2xy[num2] = np.std(gM2xy[num2+1])



C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")
